#### Packages used

In [112]:
from yahoofinancials import YahooFinancials
from hmmlearn.hmm import GaussianHMM
import statsmodels.api as sm
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import seaborn as sns

#### Download SPY, VIX data

In [113]:
start_date="2008-01-01"
end_date="2017-12-31"
time_interval="daily"

gspc = YahooFinancials("^GSPC")
gspc = gspc.get_historical_price_data(start_date=start_date, end_date=end_date, time_interval=time_interval)
gspc = pd.DataFrame(gspc["^GSPC"]['prices'])
gspc = gspc.drop('date', axis=1).set_index('formatted_date')

vix = YahooFinancials("^VIX")
vix = vix.get_historical_price_data(start_date=start_date, end_date=end_date, time_interval=time_interval)
vix = pd.DataFrame(vix["^VIX"]['prices'])
vix = vix.drop('date', axis=1).set_index('formatted_date')

In [114]:
vix_pct = vix.pct_change()[1:]
gspc_pct = gspc.pct_change()[1:]

### Charts

In [115]:
style = "none"
color_scheme = "sunset"

In [116]:
fig = px.scatter(vix_pct[["close"]], title="VIX Daily % Change", template=style, color_continuous_scale=color_scheme)
fig.show()

fig = px.line(vix, y = "close", title="VIX", template=style)
fig.show()

In [117]:
# params
nRegimes = 2
nIter = 1000

### MSDR using just VIX returns
Regime Switching signal too volatility...backtest will probably be trash

In [118]:
msdr_model_vix = sm.tsa.MarkovRegression(endog= vix_pct['close'], k_regimes=nRegimes,
    trend='c', switching_variance=True)

msdr_model_vix_results = msdr_model_vix.fit(iter=nIter)

#print model training summary
# print(msdr_model_results.summary())

marg_probs_msdr_vix = [msdr_model_vix_results.smoothed_marginal_probabilities[i] for i in range(nRegimes)]

fig = make_subplots(rows=nRegimes+1, cols = 1)
fig.update_layout(template = style)
fig.add_trace(
        go.Scatter(y = vix[1:]["close"], x = vix_pct.index, name = "VIX"),
        row = 1, col = 1
    )

for i in range(nRegimes):
    fig.add_trace(
        go.Scatter(y = marg_probs_msdr_vix[i], x = vix_pct.index, name = "VIX Regime "+str(i)), 
        row=i+2, col=1,
    )

fig.show()

C:\Users\sidiy\AppData\Local\Programs\Python\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:593: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

C:\Users\sidiy\AppData\Local\Programs\Python\Python38\lib\site-packages\statsmodels\base\optimizer.py:17: FutureWarning:

Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method bfgs is: gtol, norm, epsilon. The list of unsupported keyword arguments passed include: iter. After release 0.14, this will raise.



### Markov Switching Dynamic Regression Model
Explaining Variance in GSPC returns using Variance in VIX returns

In [119]:
msdr_model = sm.tsa.MarkovRegression(endog=gspc_pct['close'], k_regimes=nRegimes,
    trend='c', exog=vix_pct["close"], switching_variance=True)

msdr_model_results = msdr_model.fit(iter=nIter)

#print model training summary
# print(msdr_model_results.summary())

marg_probs_msdr = [msdr_model_results.smoothed_marginal_probabilities[i] for i in range(nRegimes)]

fig = make_subplots(rows=nRegimes+1, cols = 1)
fig.update_layout(template = style)
fig.add_trace(
        go.Scatter(y = gspc_pct["close"], x = gspc_pct.index, name = "GSPC % Change"),
        row = 1, col = 1
    )

for i in range(nRegimes):
    fig.add_trace(
        go.Scatter(y = marg_probs_msdr[i], x = gspc_pct.index, name = "VIX Regime "+str(i)), 
        row=i+2, col=1,
    )

fig.show()

C:\Users\sidiy\AppData\Local\Programs\Python\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:593: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

C:\Users\sidiy\AppData\Local\Programs\Python\Python38\lib\site-packages\statsmodels\base\optimizer.py:17: FutureWarning:

Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method bfgs is: gtol, norm, epsilon. The list of unsupported keyword arguments passed include: iter. After release 0.14, this will raise.



### Markov Switching Autoregressive Model
Explaining Variance in GSPC returns using Variance in VIX returns + lag=1 VIX returns

In [120]:
msar_model = sm.tsa.MarkovAutoregression(endog=gspc_pct['close'], k_regimes=nRegimes, order=2,
    trend='c', exog=vix_pct["close"], switching_variance=True)

msar_model_results = msar_model.fit(iter=nIter)

##print model training summary
# print(msar_model_results.summary())

marg_probs_msar = [msar_model_results.smoothed_marginal_probabilities[i] for i in range(nRegimes)]

fig = make_subplots(rows=nRegimes+1, cols = 1)
fig.update_layout(template = style)
fig.add_trace(
        go.Scatter(y = gspc_pct["close"], x = gspc_pct.index, name = "GSPC % Change"),
        row = 1, col = 1
    )

for i in range(nRegimes):
    fig.add_trace(
        go.Scatter(y = marg_probs_msar[i], x = gspc_pct.index, name = "VIX Regime "+str(i)), 
        row = i+2, col=1
    )

fig.show()

C:\Users\sidiy\AppData\Local\Programs\Python\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:593: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

C:\Users\sidiy\AppData\Local\Programs\Python\Python38\lib\site-packages\statsmodels\base\optimizer.py:17: FutureWarning:

Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method bfgs is: gtol, norm, epsilon. The list of unsupported keyword arguments passed include: iter. After release 0.14, this will raise.



### MSDR on just GSPC Returns

In [121]:
msdr_model_gspc = sm.tsa.MarkovRegression(gspc_pct['close'], k_regimes=nRegimes,
    trend='c', switching_variance=True)

msdr_model_gspc_results = msdr_model_gspc.fit(iter=nIter)

#print model training summary
# print(msdr_model_results.summary())

marg_probs_msdr_gspc = [msdr_model_gspc_results.smoothed_marginal_probabilities[i] for i in range(nRegimes)]

fig = make_subplots(rows=nRegimes+1, cols = 1)
fig.update_layout(template = style)
fig.add_trace(
        go.Scatter(y = gspc_pct["close"], x = gspc_pct.index, name = "GSPC % Change"),
        row = 1, col = 1
    )

for i in range(nRegimes):
    fig.add_trace(
        go.Scatter(y = marg_probs_msdr_gspc[i], x = gspc_pct.index, name = "GSPC Regime "+str(i)), 
        row=i+2, col=1,
    )

fig.show()

C:\Users\sidiy\AppData\Local\Programs\Python\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:593: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

C:\Users\sidiy\AppData\Local\Programs\Python\Python38\lib\site-packages\statsmodels\base\optimizer.py:17: FutureWarning:

Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method bfgs is: gtol, norm, epsilon. The list of unsupported keyword arguments passed include: iter. After release 0.14, this will raise.



### Hidden Markov Model
Shown that hmmlearn is same as statsmodels w/out exog vars

In [122]:
# hidden gaussian markov model
reshaped_data_vix = np.reshape(vix_pct[["close"]], [len(vix_pct[["close"]]),1])
reshaped_data_gspc = np.reshape(gspc_pct[["close"]], [len(gspc_pct[["close"]]), 1])
hgmm_model_vix = GaussianHMM(n_components=nRegimes, n_iter=nIter).fit(reshaped_data_vix)
hgmm_model_gspc = GaussianHMM(n_components=nRegimes, n_iter=nIter).fit(reshaped_data_gspc)

In [123]:
hidden_states = hgmm_model_vix.predict(reshaped_data_vix)
mu = np.array(hgmm_model_vix.means_)
sigma = np.array(np.sqrt(np.array([np.diag(hgmm_model_vix.covars_[i]) for i in range(len(hgmm_model_vix.covars_))])))
T = np.array(hgmm_model_vix.transmat_)
# find log-likelihood of HMM
prob = hgmm_model_vix.score(reshaped_data_vix)
data = vix[1:][["close"]]
data["hidden_states"] = hidden_states
px.scatter(data, x=data.index, y="close", color="hidden_states", template=style, color_continuous_scale="Emrld")

In [124]:
marg_probs_hgmm_vix = hgmm_model_vix.predict_proba(vix_pct[["close"]])
px.line(marg_probs_hgmm_vix[:, 0])

fig = make_subplots(rows=nRegimes+1, cols=1)
fig.update_layout(template = style)
fig.add_trace(
        go.Scatter(y = vix_pct["close"], x = vix_pct.index, name = "VIX % Change"),
        row = 1, col = 1
    )

for i in range(nRegimes):
    fig.add_trace(
        go.Scatter(y = marg_probs_hgmm_vix[:, i], x = gspc_pct.index, name = "GSPC Regime "+str(i)), 
        row=i+2, col=1,
    )

fig.show()
